# RAG Pipeline

## Set up
### Import Packages and API keys 

In [1]:
# !pip install transformers datasets torch langchain-community faiss-cpu sentence-transformers
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

### Model selection

In [ ]:
from langchain_community.llms import HuggingFaceHub
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct")

# I will be using T5 model from open source huggingface library
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"

model_name = "tiiuae/falcon-7b-instruct"
# model_name = "tiiuae/falcon-7b"

llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})

## RAG from synthetic data set

### Set up embeddings and documents to retrieve from

In [2]:
# Use langchain packages to help with implementing retrieval augmentation generation
from datasets import load_dataset
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser

# Step 1: Load data for retrieval
loader = CSVLoader(file_path="rag_sample.csv")
documents = loader.load()  

# Step 2: Split Documents
text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
# text_split = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
d = text_split.split_documents(documents)

# Step 3: Create a FAISS Index
modelPath = "sentence-transformers/gtr-t5-base" # Use a t5 sentence transformer model that maps sentences & paragraphs to a 768 dimensional dense vector space
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True} # Normalizing embeddings can help improve similarity metrics by ensuring that embeddings magnitude does not affect the similarity scores

# Initialize an instance of HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(d, embedding=embeddings)

# just checking
r = db.as_retriever()
docs = r.get_relevant_documents("What skills should an AI solution analyst have?")
print(docs[0].page_content)

/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Prompt: AI solutions analyst
Output: Job Description:
As an AI Solutions Analyst, you will play a crucial role in driving organizational transformations for medium- and large-scale businesses by documenting, analyzing, and improving business processes. You will work within projects to map as-is processes to to-be processes, aligning them with the future operating model. Acting as a liaison between clients and project teams, you will coordinate and collaborate with stakeholders during workshops and contribute to the design and support of ongoing solutions post-transition. Additionally, you will collaborate with teammates on the analysis and design of complex business applications using the latest technologies, ensuring successful delivery of business solutions.
Responsibilities:
Document and analyze as-is processes (functional specs and user stories) and make recommendations for improvement by mapping to-be business processes aligned with the future operating model.


### Specify model: Falcon

In [3]:
from langchain_community.llms import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate

model_name = "tiiuae/falcon-7b-instruct"
# model_name = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# llm = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

#Define a text generation pipeline using the model and tokenizer
# hf = pipeline(
#     "text-generation", 
#     model=model_name, 
#     tokenizer=tokenizer,
#     max_new_tokens = 200,
#     temperature = 0.1,
#     eos_token_id=tokenizer.eos_token_id,
#     do_sample=True
# )
# llm = HuggingFacePipeline(pipeline=hf)
llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})
# https://medium.com/international-school-of-ai-data-science/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7

# TO DELETE CACHE: huggingface-cli delete-cache

RetrievalQA

In [4]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer professionally, to the best of your ability, and where appropriate, in a Computer Science educational context.
Use the context and be specific as you can.
Context: {context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [5]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# retriever = db.as_retriever(search_kwargs={"k": 2})
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
# qa = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)
qa.verbose=True
question = "What skills should an AI solution analyst have?"
result = qa.invoke({"query": question})
print(result)
print(result["result"].split('\nHelpful Answer:')[1].strip())

# Need to be on all the documents?




> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What skills should an AI solution analyst have?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\nAnswer professionally, to the best of your ability, and where appropriate, in a Computer Science educational context.\nUse the context and be specific as you can.\nContext: Prompt: AI solutions analyst\nOutput: Job Description:\nAs an AI Solutions Analyst, you will play a crucial role in driving organizational transformations for medium- and large-scale businesses by documenting, analyzing, and improving business processes. You will work within projects to map as-is processes to to-be processes, aligning them with the future operating model. Acting as a liaison between clients and project teams, you will coordinate and collaborate with stakeholders during workshops and contribute to the design and 

In [6]:
import gradio as gr

def chat_interface(textbox, chat):
    # result = qa.invoke({'query': textbox})
    # docs = result.get("source_documents", [])
    # result=qa({"input_documents": docs,"question": query})
    # retriever = db.as_retriever()
    # docs = retriever.get_relevant_documents(textbox)
    # context = docs[0].page_content

    input_dict = {'query': textbox}
    result = qa.invoke(input_dict)
    print(result)
    text = result['result']
    answer = text.split('\nHelpful Answer:')[1].strip()
    return answer

# gr.Interface(
#     fn=chat_interface,
#     inputs=[gr.Textbox(placeholder="Ask me a question", label="Question", lines=5, type="text"),
#         gr.UploadButton(label="Upload File", file_types=[".pdf", ".csv", ".doc"])],
#     outputs=gr.Chatbot(height=300),
#     title="Chatbot",
#     description="Ask Chatbot any question",
#     theme="soft",
#     examples=[
#         ["What does AI stand for?"],
#         ["What is Software Engineering?"],
#         ["What is Cybersecurity?"]
#     ],
#     cache_examples=False,
# ).launch()

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What skills should a cybersecurity analyst have?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\nAnswer professionally, to the best of your ability, and where appropriate, in a Computer Science educational context.\nUse the context and be specific as you can.\nContext: Prompt: Cybersecurity Analyst\nOutput: Job Description:\nWe are seeking a skilled Cybersecurity Analyst to join our team. The ideal candidate will be responsible for monitoring, detecting, and responding to security incidents to protect our organization's systems and data. You will analyze security threats, conduct risk assessments, and implement security controls to mitigate vulnerabilities. The role requires strong analytical skills, attention to detail, and a deep understanding of cybersecurity principles and practices.\n\nR

With some qa Chain

In [ ]:
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer professionally, to the best of your ability, and where appropriate, in a Computer Science educational context.
Don't copy from the context.
Context: {context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
        template=template, input_variables=["context", "question"]
    )
chain_type_kwargs = {"questionPrompt": QA_CHAIN_PROMPT}

qa_chain = load_qa_chain(llm, chain_type_kwargs=chain_type_kwargs)
# qa = LLMChain(prompt=prompt, llm=llm)

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    retriever = db.as_retriever()
    docs = retriever.get_relevant_documents(textbox)
    context = docs[0].page_content

    # query = f"{context}\nQuestion: {textbox}\nHelpful Answer:"
    input_dict = {'question': textbox, 'context': context}
    result = qa.invoke(input_dict)
    print(result)
    text = result['text']
    answer = text.split('\nHelpful Answer:')[1].strip()
    return answer
    
gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

## Evaluation

In [ ]:
# RAG evaluation

### T5

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain import HuggingFacePipeline
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation", 
    model=model_name, 
    tokenizer=tokenizer,
    max_new_tokens = 200,
    temperature = 0.1,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True
)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import RetrievalQA 
retriever = db.as_retriever(search_kwargs={"k": 2})

# qa = RetrievalQA.from_chain_type(   
#   llm=llm,   
#   chain_type="refine",
#   retriever=retriever, 
#   return_source_documents=True
# )
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=False, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
print(qa)

question = "What skills should an AI solution analyst have?"
result = qa({"query": question})
print(result["result"])

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    retriever = db.as_retriever()
    docs = retriever.get_relevant_documents(textbox)
    context = docs[0].page_content

    # query = f"{context}\nQuestion: {textbox}\nHelpful Answer:"
    input_dict = {'question': textbox, 'context': context}
    result = qa.invoke(input_dict)
    print(result)
    text = result['text']
    answer = text.split('\nHelpful Answer:')[1].strip()
    return answer
    
gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    # retriever = db.as_retriever()
    # docs = r.get_relevant_documents(textbox)
    # context_str = docs[0].page_content

    # input_dict = {'question': textbox, 'context_str': context_str}
    # docs = retriever.get_relevant_documents(textbox)

    # query = f"{context}\nQuestion: {textbox}\nHelpful Answer:"
    result = qa({'query': textbox})
    docs = result.get("source_documents", [])
    print(docs)
    return result["result"]

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

In [ ]:
# MAYBE DO THIS FIRST? AND SEE THE DOWNSIDE, AND LEARN THAT IT IS NOT REQUIRED (doesn't solve hallucinations and timely context!)
# Fine-tune with input and output example data sets

# Compare with different models (one fine-tuned one just pre-trained)

## Full adapted model (combined of all approaches)

In [ ]:
# Knowledge retrieved
# Augmented Prompt
# Fine-tuned/pre-trained LLM

import gradio as gr
def chat_interface(textbox, chat):
    # docs = db.similarity_search(textbox)
    # input_dict = {'question': textbox, 'input_documents': docs }
    input_dict = {'question': textbox}
    response = llm_chain.run(input_dict)

    print("user:", textbox)
    print("bot:", response)
    return response

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()